In [2]:
# Required Libraries
import numpy as np
import pandas as pd
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD
from surprise import accuracy
import warnings
warnings.filterwarnings("ignore")

# Load Dataset Function
def load_dataset(file_name):
    df = pd.read_csv(f'{file_name}.csv', sep=';', on_bad_lines='skip', encoding='latin-1')
    return df

# Load Data
books = load_dataset("BX-Books")
users = load_dataset("BX-Users")
ratings = load_dataset("BX-Book-Ratings")

# Data Filtering
user_counts = ratings['User-ID'].value_counts()
book_counts = ratings['ISBN'].value_counts()

ratings_filtered = ratings[
    (ratings['User-ID'].isin(user_counts[user_counts > 250].index)) &
    (ratings['ISBN'].isin(book_counts[book_counts > 50].index))
]

# Merge Ratings with Books
ratings_merged = pd.merge(ratings_filtered, books[['ISBN', 'Book-Title']], on='ISBN')
ratings_merged.drop_duplicates(inplace=True)

# Setup for Recommendation Model
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(ratings_merged[['User-ID', 'Book-Title', 'Book-Rating']], reader)
trainset = data.build_full_trainset()

# Model Evaluation
models = [KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline(), SVD()]
model_performance = {}

for model in models:
    cv_scores = cross_validate(model, data, measures=['MAE', 'RMSE'], cv=5, verbose=False)
    model_name = str(model).split(" ")[0].split(".")[-1]
    model_performance[model_name] = pd.DataFrame.from_dict(cv_scores).mean(axis=0)

performance_df = pd.DataFrame(model_performance).T.sort_values(by='test_rmse')
print(performance_df[['test_rmse', 'test_mae']])

# Recommendations Function for KNN
def recommend_books_knn(user_id=13552, top_likes=5, recommendations_count=10):
    knn_model = KNNWithMeans(sim_options={'name': 'msd', 'min_support': 3, 'user_based': False})
    knn_model.fit(trainset)
    inner_user_id = trainset.to_inner_uid(user_id)
    top_books = sorted(trainset.ur[inner_user_id], key=lambda x: x[1], reverse=True)[:top_likes]
    recommendations = []

    for item_id, _ in top_books:
        neighbors = knn_model.get_neighbors(item_id, k=recommendations_count)
        recommendations.extend([trainset.to_raw_iid(inner_id) for inner_id in neighbors])
    
    return list(set(recommendations))[:recommendations_count]

# Recommendations Function for SVD
def recommend_books_svd(user_id=13552, recommendations_count=10):
    svd_model = SVD(n_factors=50, n_epochs=10, lr_all=0.005, reg_all=0.2)
    svd_model.fit(trainset)
    all_books = trainset.build_anti_testset()
    predictions = svd_model.test(all_books)
    predictions_df = pd.DataFrame(predictions)

    user_predictions = predictions_df[predictions_df['uid'] == user_id].sort_values(by="est", ascending=False)[:recommendations_count]
    recommendations = list(user_predictions['iid'])
    
    return recommendations

# Generate Recommendations
recommendations_knn = recommend_books_knn(user_id=13552, top_likes=5, recommendations_count=10)
recommendations_svd = recommend_books_svd(user_id=13552, recommendations_count=10)

print("KNN Recommendations:", recommendations_knn)
print("SVD Recommendations:", recommendations_svd)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi